In [1]:
import numpy as np
import pandas as pd

In [60]:
df1=pd.read_csv("../r2Data/user_vendor_series_data.csv",usecols=['user_id','business_id','stars_x','text','city','latitude','longitude','review_count','categories','name'],nrows=100000,keep_default_na=False)

## Buliding basic collaborative model

In [15]:
df1.drop(columns=['address','useful','funny','cool'],inplace=True)

In [33]:
df2=df1.iloc[:20000,:]

In [61]:
df1.head(3)

,user_id,business_id,stars_x,text,name,city,latitude,longitude,review_count,categories
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is...",Turning Point of North Wales,North Wales,40.210196,-75.223639,169,"Restaurants, Breakfast & Brunch, Food, Juice B..."
1,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...,Body Cycle Spinning Studio,Philadelphia,39.952103,-75.172753,144,"Active Life, Cycling Classes, Trainers, Gyms, ..."
2,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...,Kettle Restaurant,Tucson,32.207233,-110.980864,47,"Restaurants, Breakfast & Brunch"


In [62]:
pivot_table_df=pd.pivot_table(df1,index=['name'],columns=['user_id'],values='stars_x')

In [63]:
pivot_table_df.fillna(value=0,inplace=True)

In [64]:
pivot_table_df

user_id,---2PmXbF47D870stH1jqA,--4AjktZiHowEIBCMd4CZA,--E0uVPphTORm_OiZ5KCvA,--KMTwCrhKKUmr7riuS4WQ,--S8M395r8NtOCvS2LRfDw,--UizzbnQlZg7bEv2oXEyg,--WhhlIim6ISY5yhFYZreg,--_H9j6ggxvqhh9nPofZwg,--_r6E98SNIrGU7weyNxbw,--ccVMj2PN6Z9qtdOdlung,...,zzeRzizkihWHz9bVAvbcVw,zzgLgszu48af4NJULymzAA,zzgMuJgxmToqcJ5iu1TngQ,zziJLt25YU6dp01sewR-IQ,zzjThJ4A1m1N78gquSR_QA,zzljYL9c22HWRO3VvJ1_Pg,zzsPf8xNW11nd0B6MZqfRw,zzsqjDvanJhH9tn8NautOQ,zztkCqqgR6VntYbqio4UTQ,zzxUT-fFNXpp5i5xJHmvpA
name,,,,,,,,,,,,,,,,,,,,,
"""No Macarena"" DJ & Custom Music Service",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'feine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
+1 Gaming,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1 Tippling Place,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-275 Rest Area Manatee County Mile 7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
uBreakiFix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
velofix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
vybe urgent care,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
from scipy.sparse import csr_matrix

In [66]:
df_sparse=csr_matrix(pivot_table_df)

In [67]:
from sklearn.neighbors import NearestNeighbors
model1=NearestNeighbors(algorithm='brute')

In [68]:
model1.fit(df_sparse)

NearestNeighbors(algorithm='brute')

In [110]:
distance,rec=model1.kneighbors(pivot_table_df.iloc[0,:].values.reshape(1,-1),n_neighbors=20)

In [111]:
d,r=model1.kneighbors(pivot_table_df.iloc[1000,:].values.reshape(1,-1),n_neighbors=20)

In [112]:
(rec)

array([[  0, 280, 132, 118, 454, 338, 222, 102, 505, 444, 193, 183, 351,
        251, 115,  51, 514, 302, 359, 361]])

In [113]:
r

array([[1000,  280,  132,   51,  505,  115,  454,  118,  514,  444,  338,
         193,  351,  251,  183,  102,  361,  302,  222,  359]])

In [105]:
pivot_table_df.index[193]

'Action Towing'

In [135]:
def recommend(ind):
    distanceS,recS=model1.kneighbors(pivot_table_df.iloc[ind,:].values.reshape(1,-1),n_neighbors=20)
    for i in range(len(recS[0])):
        print(pivot_table_df.index[recS[0][i]])

In [136]:
recommend(1000)

Boise International Market
All 4 Dance
ADT Security Services
3am Liquor
Associated Dental Care - Tucson N Campbell
A.B. Collier Wrecker Service
Arch Engraving
A1 RV Sales
Atlantic Vapor
Aqua-Tots Swim Schools - Eagle
American Family Care
Action Towing
Americas Best Value Inn Media Philadelphia
Alamo Rent A Car
Accurate Window & Screen
A Psychic and Tarot Card Readings
Amy & Barber Beauty Salon
Aloha Japanese Steakhouse
Affordable Lock & Security Solutions - Tampa
Amor Nail and Spa


In [138]:
recommend(0)

"No Macarena" DJ & Custom Music Service
All 4 Dance
ADT Security Services
A1 RV Sales
Arch Engraving
American Family Care
Affordable Lock & Security Solutions - Tampa
A Psychic and Tarot Card Readings
Associated Dental Care - Tucson N Campbell
Aqua-Tots Swim Schools - Eagle
Action Towing
Accurate Window & Screen
Americas Best Value Inn Media Philadelphia
Alamo Rent A Car
A.B. Collier Wrecker Service
3am Liquor
Atlantic Vapor
Aloha Japanese Steakhouse
Amor Nail and Spa
Amy & Barber Beauty Salon


In [117]:
pivot_table_df.index[361]

'Amy & Barber Beauty Salon'